# Imports (compatible for Python 2 and Python 3)

In [327]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [328]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

# TASK 1

In [329]:
query = '''SELECT * FROM fb55.citibike WHERE tripduration<=180 order by start_station_id ASC, tripduration DESC '''
table = get_data(query)
print(table.head(10))
print(table.tail(10))

   the_geom        start_station_name  the_geom_webmercator  field_1  \
0       NaN          W 52 St & 11 Ave                   NaN     9059   
1       NaN  Franklin St & W Broadway                   NaN    44151   
2       NaN    St James Pl & Pearl St                   NaN     1791   
3       NaN    St James Pl & Pearl St                   NaN    37931   
4       NaN    St James Pl & Pearl St                   NaN    33591   
5       NaN    St James Pl & Pearl St                   NaN    40936   
6       NaN    St James Pl & Pearl St                   NaN     5810   
7       NaN    St James Pl & Pearl St                   NaN    43767   
8       NaN    St James Pl & Pearl St                   NaN     3787   
9       NaN    St James Pl & Pearl St                   NaN     4787   

   start_station_longitude  tripduration               starttime  \
0               -73.993929           107  2015-02-03 10:10:00+00   
1               -74.006667           149  2015-02-07 17:27:00+00   
2  

In [330]:
query1 = '''SELECT DISTINCT start_station_id FROM fb55.citibike '''
start_stations = get_data(query1)
print(start_stations)

     start_station_id  Unnamed: 1
0                 120         NaN
1                 285         NaN
2                 251         NaN
3                 195         NaN
4                 453         NaN
5                 144         NaN
6                 266         NaN
7                 263         NaN
8                 216         NaN
9                 328         NaN
10                353         NaN
11                212         NaN
12                383         NaN
13                538         NaN
14                217         NaN
15                300         NaN
16                456         NaN
17                128         NaN
18                223         NaN
19                244         NaN
20                494         NaN
21                403         NaN
22                260         NaN
23                445         NaN
24                411         NaN
25                377         NaN
26                427         NaN
27                232         NaN
28            

In [331]:
query2 = '''SELECT COUNT(DISTINCT cartodb_id) FROM fb55.citibike '''
trips = get_data(query2)
print(trips)

   count  Unnamed: 1
0  46200         NaN


In [332]:
query3 = '''SELECT 
Min(tripduration) as Min, Max(tripduration) as Max, avg(tripduration) as Avg FROM fb55.citibike 
'''
table = get_data(query3)
table

,min,max,avg
0,60,43016,675.865823


# TASK 2

In [333]:
query4 = '''SELECT * FROM fb55.citibike WHERE EXTRACT(YEAR FROM starttime)=2015 AND EXTRACT(MONTH FROM starttime)=2 AND EXTRACT(DAY FROM starttime)=2'''
table = get_data(query4)
table.head(5)


,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [334]:
queryXX = '''SELECT Avg(tripduration) as AVG, date_part('dow',starttime) FROM fb55.citibike WHERE date_part('dow',starttime)=0 OR date_part('dow',starttime)=6 GROUP BY date_part('dow',starttime)'''
table = get_data(queryXX)
table

,avg,date_part
0,686.460825,6
1,637.116968,5


In [335]:
queryXX = '''SELECT Avg(tripduration) as AVG, date_part('dow',starttime) FROM fb55.citibike WHERE date_part('dow',starttime)<>0 AND date_part('dow',starttime)<>6 GROUP BY date_part('dow',starttime)'''
table = get_data(queryXX)
table

,avg,date_part
0,638.149200,0
1,697.556559,3
2,623.396720,4
3,920.862234,1
4,767.224443,2


# TASK 3

In [336]:
query6 =  '''SELECT start_station_longitude, start_station_latitude FROM fb55.citibike GROUP BY start_station_latitude, start_station_longitude'''
table = get_data(query6)
table

,start_station_longitude,start_station_latitude
0,-73.988021,40.717227
1,-73.999733,40.719105
2,-74.004519,40.746920
3,-73.983799,40.726218
4,-73.978311,40.748238
5,-73.987895,40.764618
6,-73.977706,40.751873
7,-73.995481,40.700379
8,-73.965964,40.683178
9,-74.002150,40.730386


In [337]:
query7 =  '''SELECT COUNT(cartodb_id) as TRIPS, start_station_latitude, start_station_longitude FROM fb55.citibike GROUP BY start_station_latitude, start_station_longitude'''
table = get_data(query7)
table

,trips,start_station_latitude,start_station_longitude
0,83,40.717227,-73.988021
1,124,40.719105,-73.999733
2,202,40.746920,-74.004519
3,246,40.726218,-73.983799
4,138,40.748238,-73.978311
5,107,40.764618,-73.987895
6,323,40.751873,-73.977706
7,12,40.700379,-73.995481
8,55,40.683178,-73.965964
9,238,40.730386,-74.002150


In [338]:
# I assume that 500 metres is equal to approximately 0.00452 in latitude; TS (lat, lon):40.7577,-73.9857; Ranges: 40,76222 - 40,75318; -73,98118- -73,99022

query8 =  '''SELECT start_station_longitude, start_station_latitude FROM fb55.citibike WHERE start_station_latitude<=40.76222 AND start_station_latitude>=40.75318 AND start_station_longitude<=-73.98118 AND start_station_longitude>=-73.99022 GROUP BY start_station_latitude, start_station_longitude'''
table = get_data(query8)
table


,start_station_longitude,start_station_latitude
0,-73.986580,40.755136
1,-73.988597,40.759291
2,-73.984427,40.760647
3,-73.982912,40.756800
4,-73.990026,40.756405
5,-73.983169,40.755273


In [339]:
queryX = '''SELECT Q.start_station_id FROM
(SELECT max(tripduration) as Max, start_station_id FROM fb55.citibike
WHERE (date_part('dow',starttime)=0 OR date_part('dow',starttime)=6) AND start_station_latitude<=40.76222 AND start_station_latitude>=40.75318 AND start_station_longitude<=-73.98118 AND start_station_longitude>=-73.99022
GROUP BY start_station_id
ORDER BY Max ASC) as Q
LIMIT 1
'''
table=get_data(queryX)
table

,start_station_id,Unnamed: 1
0,465,NaN
